# 05. Going Modular: Part 1 (cell mode)

This notebook is part 1/2 of section [05. Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/).

For reference, the two parts are: 
1. [**05. Going Modular: Part 1 (cell mode)**](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/05_pytorch_going_modular_cell_mode.ipynb) - this notebook is run as a traditional Jupyter Notebook/Google Colab notebook and is a condensed version of [notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/).
2. [**05. Going Modular: Part 2 (script mode)**](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/05_pytorch_going_modular_script_mode.ipynb) - this notebook is the same as number 1 but with added functionality to turn each of the major sections into Python scripts, such as, `data_setup.py` and `train.py`. 

Why two parts?

Because sometimes the best way to learn something is to see how it *differs* from something else.

If you run each notebook side-by-side you'll see how they differ and that's where the key learnings are.

## What is cell mode?

A cell mode notebook is a regular notebook run exactly how we've been running them through the course.

Some cells contain text and others contain code.

## What's the difference between this notebook (Part 1) and the script mode notebook (Part 2)?

This notebook, 05. PyTorch Going Modular: Part 1 (cell mode), runs a cleaned up version of the most useful code from section [04. PyTorch Custom Datasets](https://www.learnpytorch.io/04_pytorch_custom_datasets/).

Running this notebook end-to-end will result in recreating the image classification model we built in notebook 04 (TinyVGG) trained on images of pizza, steak and sushi.

The main difference between this notebook (Part 1) and Part 2 is that each section in Part 2 (script mode) has an extra subsection (e.g. 2.1, 3.1, 4.1) for turning cell code into script code.

## 1. Get data

We're going to start by downloading the same data we used in [notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#1-get-data), the `pizza_steak_sushi` dataset with images of pizza, steak and sushi.

In [2]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
DATA_PATH = Path("data/")
IMG_PATH = DATA_PATH/ "pizza_steak_sushi"
ZIP_PATH = DATA_PATH/ "pizza_steak_sushi.zip"

# If the image folder doesn't exist, download it and prepare it...
if IMG_PATH.is_dir():
    print(f"{IMG_PATH} directory exists.")
else:
    print(f"Did not find {IMG_PATH} directory, creating one...")
    IMG_PATH.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open (ZIP_PATH, 'wb') as f:
    req = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(req.content)
    
# Unzip pizza, steak, sushi data
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(IMG_PATH)
    
# Delete the zip file after extraction
ZIP_PATH.unlink()
print("Deleted ZIP file after extraction.")

Did not find data\pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...
Deleted ZIP file after extraction.


In [3]:
# Setup train and testing paths
TRAIN_DIR = IMG_PATH/ "train"
TEST_DIR = IMG_PATH/ "test"

TRAIN_DIR, TEST_DIR

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

## 2. Create Datasets and DataLoaders

Now we'll turn the image dataset into PyTorch `Dataset`'s and `DataLoader`'s. 

In [4]:
from torchvision import datasets, transforms

# Create simple transform
data_trans = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

# Use ImageFolder to create dataset(s)
train_dataset = datasets.ImageFolder(TRAIN_DIR,
                                    transform= data_trans,
                                    target_transform= None)

test_dataset = datasets.ImageFolder(TEST_DIR,
                                    transform=data_trans)

print(f"Train data:\n{train_dataset}\nTest data:\n{test_dataset}")

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data\pizza_steak_sushi\train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data\pizza_steak_sushi\test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [5]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset,
                        batch_size=1,
                        num_workers=1,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                        batch_size=1,
                        num_workers=1,
                        shuffle=False)

train_loader, test_loader

(<torch.utils.data.dataloader.DataLoader at 0x27f2b032800>,
 <torch.utils.data.dataloader.DataLoader at 0x27f2b033610>)

## 3. Making a model (TinyVGG)

We're going to use the same model we used in notebook 04: TinyVGG from the CNN Explainer website.

The only change here from notebook 04 is that a docstring has been added using [Google's Style Guide for Python](https://google.github.io/styleguide/pyguide.html#384-classes). 

In [6]:
import torch
from torch import nn

class TinyVGG (nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__ (self, input_shape, hidden_units, output_shape):
        super().__init__()
        
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels= input_shape,
                    out_channels= hidden_units,
                    
                    kernel_size= 3,
                    padding=1,
                    stride=1),
            
            nn.ReLU(),
            
            nn.Conv2d(in_channels= hidden_units,
                    out_channels= hidden_units,
                    
                    kernel_size= 3,
                    padding=1,
                    stride=1),
            
            nn.ReLU(),
            
            nn.MaxPool2d(2)
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels= hidden_units,
                    out_channels= hidden_units,
                    
                    kernel_size= 3,
                    padding=1,
                    stride=1),
            
            nn.ReLU(),
            
            nn.Conv2d(in_channels= hidden_units,
                    out_channels= hidden_units,
                    
                    kernel_size= 3,
                    padding=1,
                    stride=1),
            
            nn.ReLU(),
            
            nn.MaxPool2d(2)
        )
        
        with torch.no_grad():
            # This ensures the dummy input matches the input shape expected by the model.
            temp = torch.zeros(1, input_shape, 64, 64)
            dummy = self.conv_block2(self.conv_block1(temp))
            num_features = dummy.shape[1] * dummy.shape[2] * dummy.shape[3]
            
        self.classifier = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(in_features= num_features,
                    out_features= output_shape)
        )
        
    def forward(self, x):
        return self.classifier(self.conv_block2(self.conv_block1(x)))

In [7]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                hidden_units=10, 
                output_shape=len(train_dataset.classes)).to(device)
model_0

TinyVGG(
  (conv_block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2560, out_features=3, bias=True)
  )
)

In [8]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_loader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))
    
# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[0.0578, 0.0634, 0.0351]], device='cuda:0')

Output prediction probabilities:
tensor([[0.3352, 0.3371, 0.3277]], device='cuda:0')

Output prediction label:
tensor([1], device='cuda:0')

Actual label:
2


## 4. Creating `train_step()` and `test_step()` functions and `train()` to combine them  

Rather than writing them again, we can reuse the `train_step()` and `test_step()` functions from [notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#75-create-train-test-loop-functions).

The same goes for the `train()` function we created.

The only difference here is that these functions have had docstrings added to them in [Google's Python Functions and Methods Style Guide](https://google.github.io/styleguide/pyguide.html#383-functions-and-methods).

Let's start by making `train_step()`.

In [9]:
# Install torchinfo if it's not available, import it if it is
try: 
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo
    
from torchinfo import summary
summary(model= model_0,
        input_size=[1, 3, 64, 64])

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 32, 32]           --
│    └─Conv2d: 2-1                       [1, 10, 64, 64]           280
│    └─ReLU: 2-2                         [1, 10, 64, 64]           --
│    └─Conv2d: 2-3                       [1, 10, 64, 64]           910
│    └─ReLU: 2-4                         [1, 10, 64, 64]           --
│    └─MaxPool2d: 2-5                    [1, 10, 32, 32]           --
├─Sequential: 1-2                        [1, 10, 16, 16]           --
│    └─Conv2d: 2-6                       [1, 10, 32, 32]           910
│    └─ReLU: 2-7                         [1, 10, 32, 32]           --
│    └─Conv2d: 2-8                       [1, 10, 32, 32]           910
│    └─ReLU: 2-9                         [1, 10, 32, 32]           --
│    └─MaxPool2d: 2-10                   [1, 10, 16, 16]           --
├─Sequentia

In [ ]:
from typing import Tuple
import torchmetrics 

def train_step (model: torch.nn.Module,
                dataloader: torch.utils.data.DataLoader,
                loss_fn: torch.nn.Module ,
                acc_fn: torchmetrics.Accuracy,
                optimizer: torch.optim.Optimizer,
                device: torch.device) -> Tuple[float, float]:
    
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    model.train()
    loss_total, acc_total = 0,0
    
    for batch, (x, y) in enumerate(dataloader):
        x,y = x.to(device), y.to(device)

        y_pred = model (x)
        
        loss = loss_fn(y_pred, y)
        loss_total += int(loss)
        acc_fn.update(y_pred.argmax(dim=1), y)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    # Calculate loss and accuracy per epoch and print out what's happening
    loss_total /= len(dataloader)
    acc_total = acc_fn.compute().item()
    acc_total *= 100
    acc_fn.reset()  
    
    return loss_total, acc_total   

In [13]:
import torchmetrics 

def test_step ( model: torch.nn.Module,
                dataloader: torch.utils.data.DataLoader,
                loss_fn: torch.nn.Module ,
                acc_fn: torchmetrics.Accuracy,
                device: torch.device) -> Tuple[float, float]:
    
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    
    model.eval()
    
    loss_total, acc_total = 0,0
    
    with torch.inference_mode():
        for batch, (x, y) in enumerate(dataloader):
            x,y = x.to(device), y.to(device)

            y_pred = model (x)
            
            loss = loss_fn(y_pred, y)
            loss_total += int(loss)
            acc_fn.update(y_pred.argmax(dim=1), y)
        
    # Calculate loss and accuracy per epoch and print out what's happening
    loss_total /= len(dataloader)
    acc_total = acc_fn.compute().item()
    acc_total *= 100
    acc_fn.reset()  
    
    return loss_total, acc_total    

In [20]:
from tqdm.auto import tqdm
from torchmetrics import Accuracy
import torch
from typing import Dict, List

from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
        train_dataloader: torch.utils.data.DataLoader, 
        test_dataloader: torch.utils.data.DataLoader, 
        
        optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
        
        epochs: int = 5,
        device: torch.device = 'cpu') -> Dict[str, List[float]]:
    
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
                    train_acc: [...],
                    test_loss: [...],
                    test_acc: [...]} 
    For example if training for epochs=2: 
                    {train_loss: [2.0616, 1.0537],
                    train_acc: [0.3945, 0.3945],
                    test_loss: [1.2641, 1.5706],
                    test_acc: [0.3400, 0.2973]} 
    """
    
    acc_fn = Accuracy(task="multiclass", num_classes=3).to(device)
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        acc_fn = acc_fn,
                                        device= device)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            acc_fn= acc_fn,
            device= device)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_acc"].append(train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)
        results["test_acc"].append(test_acc.item() if isinstance(test_acc, torch.Tensor) else test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

## 5. Creating a function to save the model

Let's setup a function to save our model to a directory.

In [21]:
from pathlib import Path

def save_model(model: torch.nn.Module,
                target_dir: str,
                model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
        either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
                target_dir="models",
                model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
                f=model_save_path)

## 6. Train, evaluate and save the model

Let's leverage the functions we've got above to train, test and save a model to file.


In [22]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 5

# Recreate an instance of TinyVGG
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                    hidden_units=10, 
                    output_shape=len(train_dataset.classes)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_0_results = train(model=model_0, 
                        train_dataloader=train_loader,
                        test_dataloader=test_loader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS,
                        device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model
save_model(model=model_0,
            target_dir="models",
            model_name="05_going_modular_cell_mode_tinyvgg_model.pth")

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:19<01:16, 19.01s/it]

Epoch: 1 | train_loss: 0.8178 | train_acc: 29.7778 | test_loss: 1.0000 | test_acc: 41.3333


 40%|████      | 2/5 [00:33<00:48, 16.27s/it]

Epoch: 2 | train_loss: 1.0000 | train_acc: 28.8889 | test_loss: 1.0000 | test_acc: 25.3333


...

Epoch: 3 | train_loss: 1.0000 | train_acc: 30.6667 | test_loss: 1.0000 | test_acc: 33.3333


 80%|████████  | 4/5 [00:58<00:13, 13.58s/it]

Epoch: 4 | train_loss: 1.0000 | train_acc: 32.4444 | test_loss: 1.0000 | test_acc: 33.3333


100%|██████████| 5/5 [01:08<00:00, 13.78s/it]

Epoch: 5 | train_loss: 1.0000 | train_acc: 31.1111 | test_loss: 1.0000 | test_acc: 33.3333
[INFO] Total training time: 68.912 seconds
[INFO] Saving model to: models\05_going_modular_cell_mode_tinyvgg_model.pth
